In [2]:
import h5py
%matplotlib inline
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import sys
import shutil
import glob
import optparse
import os
import json
import pandas as pd
import numpy as np
import pylab as pl
import scipy.stats as stats
import seaborn as sns
sns.set_style("ticks")
sns.set()
sns.set_color_codes()

def findOccurrences(s, ch):
    return [i for i, letter in enumerate(s) if letter == ch]

In [27]:
aggregate_root = '/n/coxfs01/cechavarria/2p-aggregate/scenes'

# sess_list = ['JC097_20190621','JC085_20190624','JC097_20190628','JC097_20190704','JC085_20190712']
# area = 'V1'

# sess_list = ['JC080_20190619','JC091_20190621','JC091_20190628','JC097_20190702','JC091_20190703','JC085_20190704','JC097_20190708']
# area = 'LM'

sess_list = ['JC091_20190625','JC091_20190701','JC091_20190705']
area = 'LI'

num_animals = 1
num_session = 3

response_type = 'norm_df'

filter_crit = 'zscore'
filter_thresh = 1

#define paths
aggregate_file_dir = os.path.join(aggregate_root,area,'files')
fig_out_dir = os.path.join(aggregate_root,area,'figures')

if not os.path.isdir(fig_out_dir):
        os.makedirs(fig_out_dir)

In [28]:
#pool all neurons


#put things into pandas df for plotting
resp_dfs = []

animalids = []
sess_count = []
active_cell_count = 0
total_cell_count = 0

ylabel = 'Average Response'

for sess_idx, sess in enumerate(sess_list):
# sess_idx = 0
# sess = sess_list[sess_idx]
    i1 = findOccurrences(sess,'_')[0]
    animalid = sess[0:i1]
    session = sess[i1+1:]


    if animalid not in animalids:
        animalids.append(animalid)
        sess_count.append(0)

    animal_idx = animalids.index(animalid)
    sess_count[animal_idx] = sess_count[animal_idx]+1

    data_array_fn = '%s_filtered_%s_responses_thresh_%s_%i.hdf5'%(sess, response_type, filter_crit, filter_thresh)
    data_array_filepath = os.path.join(aggregate_file_dir, data_array_fn)
    data_grp = h5py.File(data_array_filepath, 'r')


    curr_slice = 'Slice01'#hard-code planar data for now


    active_cell_idx = np.array(data_grp['/'.join([curr_slice, 'active_cell_idx'])])
    total_cell_count = total_cell_count + active_cell_idx.shape[0]
    active_cell_count = active_cell_count + np.nansum(active_cell_idx)
    sess_response = np.array(data_grp['/'.join([curr_slice, 'filtered_response_per_cond_per_neuron'])])

    if sess_idx == 0:
        response_array = sess_response[:,active_cell_idx]
    else:
        response_array = np.hstack((response_array,sess_response[:,active_cell_idx]))


In [29]:
fstat, anova_pval = stats.f_oneway(response_array[0,:],response_array[1,:],response_array[2,:])
print(fstat,anova_pval)

5.0849579371469575 0.006540126005837443


In [33]:
response_array[2,:]

array([1.        , 0.61563872, 0.55930246, 0.60109027, 0.27013329,
       0.460679  , 0.20470797, 0.79674646, 0.39071202, 0.50634444,
       0.59047383, 0.62720953, 0.63982689, 0.28685824, 0.44337981,
       0.92718131, 0.86468392, 0.85171664, 0.73574216, 0.70078644,
       0.41547153, 0.63263361, 0.6720514 , 0.73198397, 0.45068654,
       0.6271305 , 0.53521954, 0.58323881, 0.49092197, 0.43020187,
       0.67183756, 0.6124249 , 0.67214759, 0.74338619, 0.45756005,
       0.83025642, 0.68778586, 0.41520767, 0.48780388, 0.95618209,
       0.91228743, 0.56890873, 0.54863612, 0.38820502, 0.77434158,
       0.74655698, 0.57303949, 0.62744766, 0.48641306, 0.48260339,
       0.48117314, 0.07999139, 0.82118097, 0.74381177, 0.16775941,
       0.88028037, 0.58220883, 1.        , 0.47251617, 0.68793739,
       0.34240977, 0.28380647, 0.63527069, 0.37080995, 0.37129917,
       0.4682611 , 0.33412602, 0.40707798, 0.66509243, 0.44346294,
       0.69407192, 0.61568412, 0.56334869, 0.67958734, 0.83367

In [ ]:
data_grp.close()

In [ ]:
# 

In [ ]:
#----Junk Code below here------